In [ ]:
import pandas as pd
import os

project_path = os.path.dirname('')
data_path = project_path + "../data/"
reports_path = project_path + "../reports/"
df_ntl = pd.read_csv(data_path + "canada_national_2023.csv")
df_intl = pd.read_csv(data_path + "winners_visual.csv", index_col=0)
print("Records in NATIONAL =", len(df_intl))
# print(df_ntl)

df_intl = df_intl.query(" Country == 'CANADA' ")
print("Records in INTERNATIONAL =", len(df_intl))
# df_intl

In [ ]:
# clean data
df_ntl['Student Name'] = df_ntl['Student Name'].str.lower()
df_ntl['Placing_Num'] = df_ntl['Placing'].str.replace("Grand Champion", "1").replace("Champion", "2") \
    .replace("1st Runner up", "3").replace("2nd Runner up", "4").replace("3rd Runner up", "5") \
        .replace("4th Runner up", "6").replace("5th Runner up", "7").replace("Merit", "8")
# df_ntl['xCategory_National'] = df_ntl['Category'].str.slice(0,1)

df_intl['Student Name'] = df_intl['Student Name'].str.lower()
df_intl['Placing_Num'] = df_intl['Placing'].str.replace("Grand Champion", "1").replace("Champion", "2") \
    .replace("1st Runner Up", "3").replace("2nd Runner Up", "4").replace("3rd Runner Up", "5")

In [ ]:
# merge national and international data
df_merged = pd.merge(df_ntl, df_intl, 
              on=['Student Name'],
              suffixes=('_National', '_International'))
df_merged.to_csv(data_path + "canada_winners_merged.csv")
print("Matched Records =", len(df_merged))

In [ ]:
# find those who performed better in international (excl. 3rd runner up)
df_change = df_merged.query(" Placing_International != '3rd Runner Up' ")

df_better = df_change.query(" Placing_Num_National > Placing_Num_International ")
print("Performed Better =", len(df_better))
df_better.to_csv(reports_path+"performed_better.csv")

df_equal = df_change.query(" Placing_Num_National == Placing_Num_International ")
print("Performed Same =", len(df_equal))
df_equal.to_csv(reports_path+"performed_equal.csv")

df_worse = df_change.query(" Placing_Num_National < Placing_Num_International ")
print("Performed Worse =", len(df_worse))
df_worse.to_csv(reports_path+"performed_worse.csv")

In [ ]:
# big changes
df_merged['change'] = df_merged['Placing_Num_National'].astype(int) - df_merged['Placing_Num_International'].astype(int)
df_big_change = df_merged.query(" Placing_International != '3rd Runner Up' ")
df_big_change = df_big_change.query(" change > 1 ").sort_values('change', ascending=False)
# df_big_change = df_merged.sort_values('change', ascending=False)
df_big_change.to_csv(reports_path+"performance_change.csv")
print("Big change # moved >1 placing = ", len(df_big_change))